# Definition of DataFrame columns to "dirty" and use imputation techniques

Definição de colunas do Dataframe para "sujar" e utilizar as técnicas de imputação

### Importing the main libraries


In [1]:
import pandas as pd
import numpy as np

# problmas:
# nao tem duplicados
# precisa fazer o que foi feito no imputing, mostrando as correlaçoes de todos os tipos de insersao ou so uma e o porque dela
# devo dropar as colunas que tem pouca ou nenhuma relação para fazer o knn?
# como eu faço se a relaçao de duas variaveis apenas nao faz sentido, mas a de 3 ou 4 faça?
# usar o category cat codes no nomes dos times nao ta fazendo sentido por que a ordem é alfabeitca, entao mesmo que seja direta a correlação na tabela nao aparece
# gostaria de trabalhar com muitas colunas, tem problema, e ou dropalas para cada analise?
# hipoteses: os times que sao favoritos tem mais chutes, escanteios, chutes a gol e menos cartoes?
# no knn ele inputou 4.3 chutes a gol num time que so teve 3 finalizaçoes totais, ou seja impossivel, como eu trato?
# no projeto o objetivo vai ser analisar o passado ou estimar o futuro?
# se tem problema em guardar o index por que o merge muda a ordem, se ele sabe como fazer de outra forma?
# se é certo dropar as colunas de tempos de gols?

### CSV

In [7]:
matches_info_df = pd.read_csv("https://raw.githubusercontent.com/gabrielaragao01/Data-Science-Project/main/data/raw/england-premier-league-matches-2018-to-2019-stats.csv")
matches_info_df.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1533927600,Aug 10 2018 - 7:00pm,complete,74439,Manchester United,Leicester City,Andre Marriner,1,0.0,0.0,...,1.37,4.98,9.81,1.33,2.00,3.60,7.50,2.05,1.69,Old Trafford (Manchester)
1,1533987000,Aug 11 2018 - 11:30am,complete,51749,Newcastle United,Tottenham Hotspur,Martin Atkinson,1,0.0,0.0,...,4.51,3.77,1.84,1.31,1.95,3.50,5.75,1.83,1.87,St. James' Park (Newcastle upon Tyne)
2,1533996000,Aug 11 2018 - 2:00pm,complete,10353,AFC Bournemouth,Cardiff City,Kevin Friend,1,0.0,0.0,...,2.03,3.51,3.96,1.31,1.95,3.45,6.70,1.83,1.83,Vitality Stadium (Bournemouth- Dorset)
3,1533996000,Aug 11 2018 - 2:00pm,complete,24821,Fulham,Crystal Palace,Mike Dean,1,0.0,0.0,...,2.31,3.44,3.26,1.28,1.87,3.20,7.00,1.71,2.00,Craven Cottage (London)
4,1533996000,Aug 11 2018 - 2:00pm,complete,24121,Huddersfield Town,Chelsea,Chris Kavanagh,1,0.0,0.0,...,7.47,4.27,1.51,1.29,1.91,3.30,6.95,2.00,1.71,John Smith's Stadium (Huddersfield- West Yorks...


### Definition of the columns we will "dirty"

In [8]:
matches_info_df[['home_team_shots', 'home_team_shots_on_target']].tail(50)

,home_team_shots,home_team_shots_on_target
330,11,7
331,10,5
332,6,4
333,11,8
334,7,4
335,11,3
336,11,5
337,12,6
338,9,4
339,9,4


### Setting 30% of the "shots on target bar" data to Null and transforming them into Null data

In [9]:
# 30% das linhas
porcentagem = 0.3
num_linhas = len(matches_info_df)
num_valores_para_sujar = int(num_linhas * porcentagem)

# Definição da coluna
coluna = 'home_team_shots_on_target'

# Escolha dos valores
indices_para_sujar = np.random.choice(matches_info_df.index, num_valores_para_sujar, replace=False)

# Renomear a coluna original
matches_info_df.rename(columns={coluna: f'{coluna}_original'}, inplace=True)

# Sujar os dados na nova coluna
matches_info_df[f'{coluna}_dirty'] = matches_info_df[f'{coluna}_original']
matches_info_df.loc[indices_para_sujar, f'{coluna}_dirty'] = np.nan

# Criar uma nova coluna 'sujado' com 'no' como valor padrão
matches_info_df['dirty'] = 0

# Atualizar a coluna 'sujado' para 'yes' onde os dados foram sujados
matches_info_df.loc[indices_para_sujar, 'dirty'] = 1

num_nans = matches_info_df['home_team_shots_on_target_dirty'].isna().sum()

assert num_nans >= num_valores_para_sujar

print(num_nans)
matches_info_df.tail(50)

114


,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,home_team_shots_on_target_dirty,dirty
330,1555164000,Apr 13 2019 - 2:00pm,complete,31708,Southampton,Wolverhampton Wanderers,Jonathan Moss,34,1.13,1.25,...,3.05,1.43,2.30,4.40,11.00,1.95,1.80,St. Mary's Stadium (Southampton- Hampshire),7.0,0
331,1555173000,Apr 13 2019 - 4:30pm,complete,74478,Manchester United,West Ham United,Graham Scott,34,2.13,0.94,...,7.75,1.16,1.51,2.30,4.00,1.69,2.10,Old Trafford (Manchester),5.0,0
332,1555247100,Apr 14 2019 - 1:05pm,complete,25721,Crystal Palace,Manchester City,Martin Atkinson,34,1.00,2.13,...,1.33,1.17,1.56,2.40,3.75,1.83,1.95,Selhurst Park (London),4.0,0
333,1555255800,Apr 14 2019 - 3:30pm,complete,53279,Liverpool,Chelsea,Michael Oliver,34,2.75,1.75,...,5.25,1.28,1.87,3.25,5.50,1.83,1.95,Anfield (Liverpool),8.0,0
334,1555354800,Apr 15 2019 - 7:00pm,complete,20480,Watford,Arsenal,Craig Pawson,34,1.73,1.27,...,2.15,1.19,1.61,2.50,4.33,1.54,2.40,Vicarage Road (Watford),NaN,1
335,1555440300,Apr 16 2019 - 6:45pm,complete,30226,Brighton & Hove Albion,Cardiff City,Andre Marriner,31,1.38,0.50,...,4.65,1.42,2.30,4.35,9.00,2.00,1.77,The American Express Community Stadium (Falmer...,NaN,1
336,1555759800,Apr 20 2019 - 11:30am,complete,54489,Manchester City,Tottenham Hotspur,Michael Oliver,35,2.82,1.94,...,11.25,1.12,1.42,2.05,3.40,1.77,2.00,Etihad Stadium (Manchester),NaN,1
337,1555768800,Apr 20 2019 - 2:00pm,complete,10511,AFC Bournemouth,Fulham,David Coote,35,1.53,0.12,...,4.75,1.17,1.56,2.40,4.33,1.59,2.30,Vitality Stadium (Bournemouth- Dorset),6.0,0
338,1555768800,Apr 20 2019 - 2:00pm,complete,23957,Huddersfield Town,Watford,Roger East,35,0.47,1.18,...,1.91,1.28,1.87,3.25,6.00,1.77,2.00,John Smith's Stadium (Huddersfield- West Yorks...,NaN,1
339,1555768800,Apr 20 2019 - 2:00pm,complete,59960,West Ham United,Leicester City,Lee Probert,35,1.59,1.41,...,2.30,1.21,1.69,2.70,5.00,1.56,2.35,London Stadium (London),4.0,0


### Transforming the new data into csv

In [10]:
matches_info_df.to_csv('matches_info_df.csv')
matches_info_df

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,home_team_shots_on_target_dirty,dirty
0,1533927600,Aug 10 2018 - 7:00pm,complete,74439,Manchester United,Leicester City,Andre Marriner,1,0.00,0.00,...,9.81,1.33,2.00,3.60,7.50,2.05,1.69,Old Trafford (Manchester),7.0,0
1,1533987000,Aug 11 2018 - 11:30am,complete,51749,Newcastle United,Tottenham Hotspur,Martin Atkinson,1,0.00,0.00,...,1.84,1.31,1.95,3.50,5.75,1.83,1.87,St. James' Park (Newcastle upon Tyne),NaN,1
2,1533996000,Aug 11 2018 - 2:00pm,complete,10353,AFC Bournemouth,Cardiff City,Kevin Friend,1,0.00,0.00,...,3.96,1.31,1.95,3.45,6.70,1.83,1.83,Vitality Stadium (Bournemouth- Dorset),5.0,0
3,1533996000,Aug 11 2018 - 2:00pm,complete,24821,Fulham,Crystal Palace,Mike Dean,1,0.00,0.00,...,3.26,1.28,1.87,3.20,7.00,1.71,2.00,Craven Cottage (London),7.0,0
4,1533996000,Aug 11 2018 - 2:00pm,complete,24121,Huddersfield Town,Chelsea,Chris Kavanagh,1,0.00,0.00,...,1.51,1.29,1.91,3.30,6.95,2.00,1.71,John Smith's Stadium (Huddersfield- West Yorks...,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1557669600,May 12 2019 - 2:00pm,complete,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,...,9.00,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool),6.0,0
376,1557669600,May 12 2019 - 2:00pm,complete,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,...,10.00,1.12,1.41,2.00,3.60,1.74,2.05,Old Trafford (Manchester),11.0,0
377,1557669600,May 12 2019 - 2:00pm,complete,30367,Southampton,Huddersfield Town,Lee Probert,38,1.22,0.33,...,7.50,1.19,1.61,2.50,4.75,1.80,2.00,St. Mary's Stadium (Southampton- Hampshire),4.0,0
378,1557669600,May 12 2019 - 2:00pm,complete,60124,Tottenham Hotspur,Everton,Andre Marriner,38,2.06,1.06,...,3.55,1.22,1.71,2.80,6.00,1.59,2.30,Tottenham Hotspur Stadium (London),4.0,0
